###  Spotify-YouTube 음악 콘텐츠 최적화 전략 수립

In [3]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from scipy import stats
import mlflow
import mlflow.sklearn
import warnings
import os
from datetime import datetime
import json
import pickle
import logging
warnings.filterwarnings('ignore')


In [4]:

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [5]:
class SpotifyYouTubeAnalytics:
    def __init__(self, csv_file=None):
        self.df = None
        self.models = {}
        self.kpis = {}
        self.conn = None
        self.feature_importance = {}
        self.model_metrics = {}
        
        self._create_directories()
        
        # MLflow 설정
        self._setup_mlflow()
        
        if csv_file:
            self.load_and_preprocess_data(csv_file)
    
    def _create_directories(self):
        """필요한 디렉토리들을 미리 생성"""
        directories = ['models', 'reports', 'data', 'mlruns']
        for directory in directories:
            os.makedirs(directory, exist_ok=True)
            logger.info(f"디렉토리 생성/확인: {directory}")
    
    def _setup_mlflow(self):
        """MLflow 설정"""
        mlflow.set_tracking_uri("./mlruns")
        try:
            mlflow.set_experiment("Spotify_YouTube_Analytics")
        except:
            mlflow.create_experiment("Spotify_YouTube_Analytics")
            mlflow.set_experiment("Spotify_YouTube_Analytics")
        
    def load_and_preprocess_data(self, csv_file):
        """데이터 로드 및 전처리"""        
        try:
            if not os.path.exists(csv_file):
                raise FileNotFoundError(f"파일을 찾을 수 없습니다: {csv_file}")
            
            self.df = pd.read_csv(csv_file)
            logger.info(f"데이터셋 크기: {self.df.shape}")
            
            self.df.columns = self.df.columns.str.strip()
            
            numeric_cols = ['Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 
                           'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 
                           'Tempo', 'Duration_ms', 'Stream', 'Views', 'Likes', 'Comments']
            
            for col in numeric_cols:
                if col in self.df.columns:
                    self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
            
            if 'Licensed' in self.df.columns:
                self.df['Licensed'] = self.df['Licensed'].astype(bool)
            if 'official_video' in self.df.columns:
                self.df['official_video'] = self.df['official_video'].fillna(False).astype(bool)
            
            self.df['Stream'] = self.df['Stream'].fillna(0)
            self.df['Views'] = self.df['Views'].fillna(0) 
            self.df['Likes'] = self.df['Likes'].fillna(0)
            self.df['Comments'] = self.df['Comments'].fillna(0)
            
            audio_features = ['Danceability', 'Energy', 'Loudness', 'Speechiness', 
                             'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']
            for col in audio_features:
                if col in self.df.columns:
                    self.df[col] = self.df[col].fillna(self.df[col].median())
            
            original_size = len(self.df)
            self.df = self._remove_outliers(self.df, ['Stream', 'Views', 'Likes'], threshold=3)
            logger.info(f"이상치 제거: {original_size} → {len(self.df)} 행")
            
            self._feature_engineering()
            
            # SQLite 데이터베이스 생성
            self.conn = sqlite3.connect('data/spotify_analytics.db')
            self.df.to_sql('tracks', self.conn, if_exists='replace', index=False)
            
            logger.info(f" 전처리 완료 - 최종 데이터 크기: {self.df.shape}")
            
        except Exception as e:
            logger.error(f" 데이터 로드 중 오류: {str(e)}")
            raise
    
    def _remove_outliers(self, df, columns, threshold=3):
        """이상치 제거"""
        for col in columns:
            if col in df.columns:
                z_scores = np.abs(stats.zscore(df[col]))
                df = df[z_scores < threshold]
        return df
    
    def _feature_engineering(self):
        """피처 엔지니어링"""
    
        # 새로운 피처 생성
        self.df['duration_minutes'] = self.df['Duration_ms'] / 60000
        self.df['audio_positivity'] = (self.df['Danceability'] + self.df['Energy'] + self.df['Valence']) / 3
        self.df['audio_complexity'] = self.df['Speechiness'] + self.df['Instrumentalness'] + self.df['Acousticness']
        self.df['youtube_engagement'] = (self.df['Likes'] + self.df['Comments']) / (self.df['Views'] + 1)
        self.df['cross_platform_ratio'] = self.df['Stream'] / (self.df['Views'] + 1)
        self.df['engagement_rate'] = self.df['Likes'] / (self.df['Views'] + 1)
        self.df['comment_rate'] = self.df['Comments'] / (self.df['Views'] + 1)
        
        # 로그 변환
        self.df['log_streams'] = np.log1p(self.df['Stream'])
        self.df['log_views'] = np.log1p(self.df['Views'])
    
    def calculate_comprehensive_kpis(self):
        """전사 KPI 계산"""
        logger.info(" KPI 계산")
        
        self.kpis = {
            # 핵심 비즈니스 메트릭
            'total_tracks': len(self.df),
            'total_artists': self.df['Artist'].nunique(),
            'total_streams': self.df['Stream'].sum(),
            'total_views': self.df['Views'].sum(),
            'avg_streams_per_track': self.df['Stream'].mean(),
            'median_streams': self.df['Stream'].median(),
            
            # 콘텐츠 다양성 지표
            'artist_diversity': self.df['Artist'].nunique() / len(self.df),
            
            # 크로스 플랫폼 성과
            'cross_platform_correlation': self.df[(self.df['Stream'] > 0) & (self.df['Views'] > 0)]['Stream'].corr(
                self.df[(self.df['Stream'] > 0) & (self.df['Views'] > 0)]['Views']
            ),
            
            # 참여도 지표
            'avg_engagement_rate': self.df['engagement_rate'].mean(),
            'avg_comment_rate': self.df['comment_rate'].mean(),
            
            # 오디오 특성 트렌드
            'avg_danceability': self.df['Danceability'].mean(),
            'avg_energy': self.df['Energy'].mean(),
            'avg_valence': self.df['Valence'].mean(),
            'avg_tempo': self.df['Tempo'].mean(),
        }
        
        # 오피셜 비디오 임팩트
        if 'official_video' in self.df.columns:
            official_streams = self.df[self.df['official_video'] == True]['Stream'].mean()
            unofficial_streams = self.df[self.df['official_video'] == False]['Stream'].mean()
            self.kpis['official_video_lift'] = (official_streams / unofficial_streams) - 1 if unofficial_streams > 0 else 0
        
        return self.kpis
    
    def build_prediction_models_with_mlflow(self):
        """MLflow를 활용한 예측 모델 구축"""
        logger.info(" MLflow와 함께 예측 모델 구축")
        
        # 실험 메타데이터
        experiment_info = {
            'dataset_size': len(self.df),
            'features_count': len(self.df.columns),
            'timestamp': datetime.now().isoformat(),
            'data_quality_score': (1 - self.df.isnull().sum().sum() / (len(self.df) * len(self.df.columns))) * 100
        }
        
        with mlflow.start_run(run_name=f"Spotify_Analytics_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
            mlflow.log_params(experiment_info)
            
            # 1. 스트림 예측 모델
            stream_model, stream_metrics = self._build_stream_prediction_model()
            self.models['stream_model'] = stream_model
            self.model_metrics.update(stream_metrics)
            
            # 2. 히트 예측 모델  
            hit_model, hit_metrics = self._build_hit_prediction_model()
            self.models['hit_model'] = hit_model
            self.model_metrics.update(hit_metrics)
            
            # 3. YouTube 성과 예측 모델
            youtube_model, youtube_metrics = self._build_youtube_prediction_model()
            self.models['youtube_model'] = youtube_model
            self.model_metrics.update(youtube_metrics)
            
            # 전체 메트릭 로깅
            mlflow.log_metrics(self.model_metrics)
            
            stream_features = [
                'Danceability', 'Energy', 'Valence', 'Tempo', 'Loudness',
                'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
                'duration_minutes', 'audio_positivity', 'audio_complexity',
                'log_views', 'youtube_engagement'
            ]
            input_example = self.df[stream_features].iloc[:5].fillna(0)
            
            # 모델 아티팩트 저장 
            mlflow.sklearn.log_model(
            stream_model, 
            name="stream_model",  
            input_example=input_example,
            signature=mlflow.models.infer_signature(input_example, stream_model.predict(input_example))
            )
            
            hit_features = [
                'Danceability', 'Energy', 'Valence', 'Tempo', 'Loudness',
                'audio_positivity', 'duration_minutes', 'youtube_engagement'
            ]
            hit_input_example = self.df[hit_features].iloc[:5].fillna(0)
            
            mlflow.sklearn.log_model(
                hit_model,
                name="hit_model",
                input_example=hit_input_example,
                signature=mlflow.models.infer_signature(hit_input_example, hit_model.predict(hit_input_example))
            )
            
            youtube_features = [
                'Danceability', 'Energy', 'Valence', 'Tempo', 
                'audio_positivity', 'duration_minutes', 'log_streams'
            ]
            valid_data = self.df[self.df['Views'] > 0]
            youtube_input_example = valid_data[youtube_features].iloc[:5].fillna(0)
            
            mlflow.sklearn.log_model(
                youtube_model,
                name="youtube_model",
                input_example=youtube_input_example,
                signature=mlflow.models.infer_signature(youtube_input_example, youtube_model.predict(youtube_input_example))
            )
            
            # 피처 중요도 저장
            self.feature_importance = self._get_feature_importance()
            mlflow.log_dict(self.feature_importance, "feature_importance.json")
            
            # 모델을 pickle로도 저장 (배포용)
            with open('models/stream_model.pkl', 'wb') as f:
                pickle.dump(stream_model, f)
            with open('models/hit_model.pkl', 'wb') as f:
                pickle.dump(hit_model, f)
            with open('models/youtube_model.pkl', 'wb') as f:
                pickle.dump(youtube_model, f)
            
            # 모델 메타데이터 저장
            model_metadata = {
                'models': list(self.models.keys()),
                'metrics': self.model_metrics,
                'feature_importance': self.feature_importance,
                'training_timestamp': datetime.now().isoformat()
            }
            
            with open('models/model_metadata.json', 'w') as f:
                json.dump(model_metadata, f, indent=2)
            
            logger.info(" MLflow 모델 기록 완료!")
            
        return self.models
    
    def _build_stream_prediction_model(self):
        """스트림 예측 모델 구축"""
        stream_features = [
            'Danceability', 'Energy', 'Valence', 'Tempo', 'Loudness',
            'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
            'duration_minutes', 'audio_positivity', 'audio_complexity',
            'log_views', 'youtube_engagement'
        ]
        
        X = self.df[stream_features].fillna(0)
        y = self.df['log_streams']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Random Forest 모델
        model = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42)
        model.fit(X_train, y_train)
        
        # 예측 및 평가
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        # Cross-validation
        cv_scores = cross_val_score(model, X, y, cv=3, scoring='r2')
        
        metrics = {
            'stream_r2': r2,
            'stream_mse': mse,
            'stream_cv_mean': cv_scores.mean(),
            'stream_cv_std': cv_scores.std()
        }
        
        # MLflow 파라미터 로깅
        mlflow.log_params({
            'stream_n_estimators': 200,
            'stream_max_depth': 15,
            'stream_features_count': len(stream_features)
        })
        
        return model, metrics
    
    def _build_hit_prediction_model(self):
        """히트 예측 모델 구축"""
        # 히트 정의 (상위 10%)
        self.df['is_hit'] = (self.df['Stream'] > self.df['Stream'].quantile(0.9)).astype(int)
        
        hit_features = [
            'Danceability', 'Energy', 'Valence', 'Tempo', 'Loudness',
            'audio_positivity', 'duration_minutes', 'youtube_engagement'
        ]
        
        X = self.df[hit_features].fillna(0)
        y = self.df['is_hit']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        report = classification_report(y_test, y_pred, output_dict=True)
        
        metrics = {
            'hit_accuracy': report['accuracy'],
            'hit_precision': report['weighted avg']['precision'],
            'hit_recall': report['weighted avg']['recall'],
            'hit_f1': report['weighted avg']['f1-score']
        }
        
        mlflow.log_params({
            'hit_n_estimators': 200,
            'hit_max_depth': 10,
            'hit_threshold': 0.9
        })
        
        return model, metrics
    
    def _build_youtube_prediction_model(self):
        """YouTube 성과 예측 모델 구축"""
        youtube_features = [
            'Danceability', 'Energy', 'Valence', 'Tempo', 
            'audio_positivity', 'duration_minutes', 'log_streams'
        ]
        
        valid_data = self.df[self.df['Views'] > 0]
        X = valid_data[youtube_features].fillna(0)
        y = valid_data['log_views']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestRegressor(n_estimators=150, max_depth=12, random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        metrics = {
            'youtube_r2': r2,
            'youtube_mse': mse
        }
        
        mlflow.log_params({
            'youtube_n_estimators': 150,
            'youtube_max_depth': 12
        })
        
        return model, metrics
    
    def _get_feature_importance(self):
        """피처 중요도 계산"""
        if 'stream_model' in self.models:
            stream_features = [
                'Danceability', 'Energy', 'Valence', 'Tempo', 'Loudness',
                'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
                'duration_minutes', 'audio_positivity', 'audio_complexity',
                'log_views', 'youtube_engagement'
            ]
            
            importance_dict = dict(zip(
                stream_features,
                self.models['stream_model'].feature_importances_.tolist()
            ))
            
            return importance_dict
        return {}
    
    def generate_executive_report(self):
        logger.info(" 경영진 리포트 생성 중...")
        
        report = {
            'timestamp': datetime.now().isoformat(),
            'dataset_overview': {
                'total_tracks': len(self.df),
                'unique_artists': self.df['Artist'].nunique(),
                'total_streams': int(self.df['Stream'].sum()),
                'total_views': int(self.df['Views'].sum()),
                'data_quality_score': (1 - self.df.isnull().sum().sum() / (len(self.df) * len(self.df.columns))) * 100
            },
            'performance_metrics': self.kpis,
            'model_performance': self.model_metrics
        }
        
        with open('reports/executive_report.json', 'w') as f:
            json.dump(report, f, indent=2)
        
      
        
        print(f" 데이터셋:")
        print(f"   • 총 트랙 수: {report['dataset_overview']['total_tracks']:,}")
        print(f"   • 총 아티스트 수: {report['dataset_overview']['unique_artists']:,}")
        print(f"   • 총 스트림 수: {report['dataset_overview']['total_streams']:,}")
        print(f"   • 총 조회수: {report['dataset_overview']['total_views']:,}")
        print(f"   • 데이터 품질 점수: {report['dataset_overview']['data_quality_score']:.1f}%")
        print()
        
        print(f" 핵심 비즈니스 지표:")
        print(f"   • 평균 스트림/트랙: {self.kpis.get('avg_streams_per_track', 0):,.0f}")
        print(f"   • 크로스 플랫폼 상관관계: {self.kpis.get('cross_platform_correlation', 0):.3f}")
        print(f"   • 평균 참여율: {self.kpis.get('avg_engagement_rate', 0)*100:.2f}%")
        print(f"   • 아티스트 다양성 지수: {self.kpis.get('artist_diversity', 0):.3f}")
        if 'official_video_lift' in self.kpis:
            print(f"   • 공식 비디오 효과: +{self.kpis['official_video_lift']*100:.1f}%")
        print()
        
        print(f" 모델 성능:")
        if self.model_metrics:
            print(f"   • 스트림 예측 R²: {self.model_metrics.get('stream_r2', 0):.3f}")
            print(f"   • 히트 예측 정확도: {self.model_metrics.get('hit_accuracy', 0)*100:.1f}%")
            print(f"   • YouTube 예측 R²: {self.model_metrics.get('youtube_r2', 0):.3f}")
        print()
        
        print(f" 오디오 트렌드:")
        print(f"   • 평균 댄서빌리티: {self.kpis.get('avg_danceability', 0):.3f}")
        print(f"   • 평균 에너지: {self.kpis.get('avg_energy', 0):.3f}")
        print(f"   • 평균 감정가: {self.kpis.get('avg_valence', 0):.3f}")
        print(f"   • 평균 템포: {self.kpis.get('avg_tempo', 0):.0f} BPM")
        print("=" * 60)
        
        return report
    
    def run_complete_analysis(self):
        logger.info(" 전체 분석 파이프라인 시작")
        
        start_time = datetime.now()
        
        try:
            # 1. KPI 계산
            self.calculate_comprehensive_kpis()
            
            # 2. 예측 모델 구축
            self.build_prediction_models_with_mlflow()
            
            # 3. 경영진 리포트
            report = self.generate_executive_report()
            
            self.df.to_csv('data/processed_spotify_youtube.csv', index=False)
            
            end_time = datetime.now()
            execution_time = (end_time - start_time).total_seconds()
            
            # 최종 실행 요약
            execution_summary = {
                'start_time': start_time.isoformat(),
                'end_time': end_time.isoformat(),
                'execution_time_seconds': execution_time,
                'status': 'SUCCESS',
                'models_created': len(self.models),
                'data_rows': len(self.df)
            }
            
            with open('reports/execution_summary.json', 'w') as f:
                json.dump(execution_summary, f, indent=2)
            
            logger.info(f" 전체 분석 완료! 실행 시간: {execution_time:.2f}초")
            
            
            print(f"  실행 시간: {execution_time:.2f}초")
            print(f" 생성된 모델: {len(self.models)}개")
            print(f" 처리된 데이터: {len(self.df):,}행")
            print("\n 생성된 파일:")
            print("   • models/ - 머신러닝 모델")
            print("   • reports/ - 분석 리포트")
            print("   • data/ - 처리된 데이터")
            print("   • mlruns/ - MLflow 실험 기록")
            print("\n next:")
            print("   • Streamlit 대시보드: streamlit run dashboard.py")
            print("   • MLflow UI: mlflow ui")
            print("="*60)
            
            return execution_summary
            
        except Exception as e:
            error_summary = {
                'start_time': start_time.isoformat(),
                'error_time': datetime.now().isoformat(),
                'status': 'ERROR',
                'error_message': str(e),
                'execution_time_seconds': (datetime.now() - start_time).total_seconds()
            }
            
            logger.error(f" 분석 중 오류 발생: {str(e)}")
            
            with open('reports/error_log.json', 'w') as f:
                json.dump(error_summary, f, indent=2)
            
            raise
        
        finally:
            if self.conn:
                self.conn.close()

In [6]:
def main():
    try:
        
        csv_file = r'C:\Users\color\Desktop\KPI\Spotify_Youtube.csv'
        
        
        if not os.path.exists(csv_file):
            print(f" 파일을 찾을 수 없습니다: {csv_file}")
            print("파일 경로를 확인하세요.")
            return
        
        analyzer = SpotifyYouTubeAnalytics(csv_file)
        summary = analyzer.run_complete_analysis()
        
        print(f"자세한 결과는 MLflow UI : mlflow ui")
        
        return summary
        
    except Exception as e:
        logger.error(f" 실행 중 오류: {str(e)}")
        print(f"오류 발생: {str(e)}")
        print("자세한 오류 로그는 reports/error_log.json")

if __name__ == "__main__":
    main()

2025-10-31 21:34:51,523 - INFO - 디렉토리 생성/확인: models
2025-10-31 21:34:51,526 - INFO - 디렉토리 생성/확인: reports
2025-10-31 21:34:51,527 - INFO - 디렉토리 생성/확인: data
2025-10-31 21:34:51,529 - INFO - 디렉토리 생성/확인: mlruns


Spotify-YouTube 통합 분석 시스템 시작


2025-10-31 21:34:51,991 - INFO - 데이터셋 크기: (20718, 28)
2025-10-31 21:34:52,028 - INFO - 이상치 제거: 20718 → 19376 행
2025-10-31 21:34:52,409 - INFO -  전처리 완료 - 최종 데이터 크기: (19376, 37)
2025-10-31 21:34:52,410 - INFO -  전체 분석 파이프라인 시작
2025-10-31 21:34:52,411 - INFO -  KPI 계산
2025-10-31 21:34:52,430 - INFO -  MLflow와 함께 예측 모델 구축


2025-10-31 21:38:05,996 - INFO -  MLflow 모델 기록 완료!
2025-10-31 21:38:06,002 - INFO -  경영진 리포트 생성 중...


 데이터셋:
   • 총 트랙 수: 19,376
   • 총 아티스트 수: 2,076
   • 총 스트림 수: 1,774,355,067,032
   • 총 조회수: 874,786,101,040
   • 데이터 품질 점수: 99.7%

 핵심 비즈니스 지표:
   • 평균 스트림/트랙: 91,574,890
   • 크로스 플랫폼 상관관계: 0.456
   • 평균 참여율: 1.21%
   • 아티스트 다양성 지수: 0.107
   • 공식 비디오 효과: +31.4%

 모델 성능:
   • 스트림 예측 R²: 0.127
   • 히트 예측 정확도: 89.7%
   • YouTube 예측 R²: 0.351

 오디오 트렌드:
   • 평균 댄서빌리티: 0.616
   • 평균 에너지: 0.632
   • 평균 감정가: 0.530
   • 평균 템포: 121 BPM


2025-10-31 21:38:06,860 - INFO -  전체 분석 완료! 실행 시간: 194.45초


  실행 시간: 194.45초
 생성된 모델: 3개
 처리된 데이터: 19,376행

 생성된 파일:
   • models/ - 머신러닝 모델
   • reports/ - 분석 리포트
   • data/ - 처리된 데이터
   • mlruns/ - MLflow 실험 기록

 next:
   • Streamlit 대시보드: streamlit run dashboard.py
   • MLflow UI: mlflow ui

 Analy
자세한 결과는 MLflow UI : mlflow ui
